## 1. Import or define trained keras model

Define Pytorch model

In [ ]:
import torch
from torch import nn
from torchvision import transforms

class Net(nn.Module):
    def __init__(self, num_classes):
        super(Net, self).__init__()
        self.conv_layer1 = nn.Conv2d(in_channels=3, out_channels=18, kernel_size=(3,13))
        self.batch1 = nn.BatchNorm2d(18)
        self.conv_layer2 = nn.Conv2d(in_channels=18, out_channels=32, kernel_size=(3,13))
        self.batch2 = nn.BatchNorm2d(32)
        self.conv_layer3 = nn.Conv2d(in_channels=32, out_channels=32, kernel_size=(3,13))
    
        self.sig = nn.Sigmoid()
        self.fc1 = nn.Linear(79872, 164)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(164, num_classes)
    
    def forward(self, x):
        r = 0; c = 0; n = 0

        x = self.conv_layer1(x)
        print(x.shape)
        print(x[0][0][0][0])
        print(x[0][1][0][0])
        print(x[0][2][0][0])
        x = self.batch1(x)
        x = self.conv_layer2(x)
        x = self.batch2(x)
        x = self.conv_layer3(x)

        x = x.reshape(x.size(0), -1)
        x = self.sig(x)
        x = self.fc1(x)
        x = self.sig(x)
        x = self.fc2(x)
        return x

model = Net(2)

model.eval()





## 2. info about c++ model

In [109]:
import os
WORKSPACE = os.path.dirname(os.getcwd())
print(WORKSPACE)

modelpath = WORKSPACE+'/pythonAPI/saved_model_binary.pth'

model_name   = 'binary' 
filename     = WORKSPACE+'/examples/' + model_name + '/model/dcnn_' + model_name + '.h'
filename_tb  = WORKSPACE+'/examples/' + model_name + '/dcnn_' + model_name + '_tb.cpp'
in_namespace = True
namespace    = 'dcnn'
second_namespace = 'cpp' # 'cpp' or 'hls'
headerDef    = '__DCNN_MODEL__'
include_list = ['"defines.h"', '"conv2d.h"', '"normalize.h"',
                '"pool2d.h"', '"dense.h"', '"flatten.h"', 
                '<mc_scverify.h>']

interface = 'VECTOR' if second_namespace == 'cpp' else 'CHANNEL'

/mnt/c/Users/dionisis/OneDrive - Democritus University of Thrace/Workspace/duth_cnn_model


In [110]:
if not os.path.exists(WORKSPACE+'/examples/' + model_name):
  os.makedirs(WORKSPACE+'/examples/' + model_name)
  os.makedirs(WORKSPACE+'/examples/' + model_name + '/model')
  os.makedirs(WORKSPACE+'/examples/' + model_name + '/parameters')
  os.makedirs(WORKSPACE+'/examples/' + model_name + '/parameters/weights')
  os.makedirs(WORKSPACE+'/examples/' + model_name + '/parameters/biases')
else:
  if not os.path.exists(WORKSPACE+'/examples/' + model_name + '/model'):
    os.makedirs(WORKSPACE+'/examples/' + model_name + '/model')
  if not os.path.exists(WORKSPACE+'/examples/' + model_name + '/parameters'):
    os.makedirs(WORKSPACE+'/examples/' + model_name + '/parameters')
    os.makedirs(WORKSPACE+'/examples/' + model_name + '/parameters/weights')
    os.makedirs(WORKSPACE+'/examples/' + model_name + '/parameters/biases')
  else:
    if not os.path.exists(WORKSPACE+'/examples/' + model_name + '/parameters/weights'):
      os.makedirs(WORKSPACE+'/examples/' + model_name + '/parameters/weights')
    if not os.path.exists(WORKSPACE+'/examples/' + model_name + '/parameters/biases'):
      os.makedirs(WORKSPACE+'/examples/' + model_name + '/parameters/biases')

## 3. Convert Keras or PyTorch model to C++
For a Keras/Pytorch model import from_keras/from_torch respectively.

In [111]:
import write_dcnn 

lib = "torch" # "keras" ορ "torch"

if lib == "keras":
  from export_model import from_keras as exporter
elif lib == "torch":
  from export_model import from_torch as exporter

layers_list, trainable_layers = exporter.export(model)

model_parameters = {}
model_parameters['workspace']    = WORKSPACE
model_parameters['name']         = model_name
model_parameters['filename']     = filename
model_parameters['filename_tb']  = filename_tb
model_parameters['in_namespace'] = in_namespace
model_parameters['namespace']    = namespace
model_parameters['namespace_2']  = second_namespace
model_parameters['interface']    = interface
model_parameters['headerDef']    = headerDef
model_parameters['includes']     = include_list
model_parameters['layers']       = layers_list
model_parameters['trainable']    = trainable_layers



write_dcnn.defineModel(model_parameters)

## 4. Export parammeters of the model 
Model trainable parameters are weights, biases, scaling factors (for normalization and/or quantization)

In [ ]:
if lib == 'keras':
  from export_model import get_features_weights as extract
else:
  from export_model import get_features_weights_torch as extract

path = WORKSPACE+'/examples/' + model_name + '/parameters'
if not os.path.exists(path):
  os.mkdir(path)

extract.export_all_w_b(model, path)